### Passo 1 - Importar Arquivos e Bibliotecas

In [1]:
import pandas as pd
import pathlib 

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
#Importar e tratar base de dados
#Criar 1 arquivo para cada loja
#Salvar o backup nas pastas
#Calcular os indicadores
#Enviar OnePage
#Enviar email para diretoria

In [5]:
df_vendas = pd.read_excel(r'C:\Users\Pichau\Documents\Python Scripts\py-treinamento-hashtag-main\proj_automacao_processos\Projeto AutomacaoIndicadores\Bases de Dados\Vendas.xlsx')
df_emails = pd.read_excel(r'C:\Users\Pichau\Documents\Python Scripts\py-treinamento-hashtag-main\proj_automacao_processos\Projeto AutomacaoIndicadores\Bases de Dados\Emails.xlsx')
df_lojas = pd.read_csv(r'C:\Users\Pichau\Documents\Python Scripts\py-treinamento-hashtag-main\proj_automacao_processos\Projeto AutomacaoIndicadores\Bases de Dados\Lojas.csv',encoding='latin1',sep=";")
display(df_emails.head())
display(df_vendas.head())
display(df_lojas.head())

df_vendas = df_vendas.merge(df_lojas,on="ID Loja")
df_vendas


,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,kaio.alves123+Helena@gmail.com
1,Shopping Midway Mall,Alice,kaio.alves123+Alice@gmail.com
2,Norte Shopping,Laura,kaio.alves123+Laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,kaio.alves123+Manuela@gmail.com
4,Shopping União de Osasco,Valentina,kaio.alves123+Valentina@gmail.com


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71


,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,2,2019-01-02,3,Relógio,3,200,600,Norte Shopping
4,2,2019-01-02,3,Chinelo Liso,1,71,71,Norte Shopping
...,...,...,...,...,...,...,...,...
100994,69996,2019-12-26,17,Short Listrado,2,102,204,Center Shopping Uberlândia
100995,69996,2019-12-26,17,Mochila,4,270,1080,Center Shopping Uberlândia
100996,69996,2019-12-26,17,Pulseira Estampa,1,87,87,Center Shopping Uberlândia
100997,69997,2019-12-26,11,Camisa Listrado,1,108,108,Ribeirão Shopping


### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [6]:
dic_lojas = {}

for loja in df_lojas['Loja']:
    dic_lojas[loja] = df_vendas.loc[df_vendas["Loja"]==loja,:]


In [9]:
dia_indicador = df_vendas["Data"].max()


### Passo 3 - Salvar a planilha na pasta de backup

In [14]:
#Identificar se as pastas existem, criar e depois salvar os arquivos separadamente
path_backup = pathlib.Path(f'Backup Arquivos Lojas')
path_backup.iterdir()
pastas = [arquivo.name for arquivo in path_backup.iterdir()]

for loja in dic_lojas:
    if loja not in pastas:
        path_lojas = path_backup / loja
        path_lojas.mkdir()
    #Salvar arquivos em massa
    nome_arquivo = f'{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx'
    path_arquivo = path_backup / loja / nome_arquivo
    dic_lojas[loja].to_excel(path_arquivo)


### Passo 4 - Calcular o indicador para 1 loja

In [11]:
#Definir metas
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtd_produtos_dia = 4
meta_qtd_produtos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

In [12]:
# Configurações do servidor SMTP do Gmail
smtp_host = 'smtp.gmail.com'
smtp_port = 587
login = 'kaio.alves123@gmail.com'
senha = 'znwc ivdb xiyo lrae'
# Configurações do e-mail

In [15]:
i=0
for loja in dic_lojas:
    i+=1
    df_vendas_loja = dic_lojas[loja]
    df_vendas_loja_dia = df_vendas_loja.loc[df_vendas_loja['Data']==dia_indicador, :]
    #Faturamento
    faturamento_ano = df_vendas_loja["Valor Final"].sum()
    faturamento_dia = df_vendas_loja_dia["Valor Final"].sum()


    #Diversidade produtos
    qtd_produtos_ano = len(df_vendas_loja["Produto"].unique())
    qtd_produtos_dia = len(df_vendas_loja_dia["Produto"].unique())


    #Ticket Medio Ano e dia
    ticket_medio_ano = df_vendas_loja.groupby(r'Código Venda').sum(numeric_only=True)
    ticket_medio_ano = ticket_medio_ano['Valor Final'].mean()
    ticket_medio_dia = df_vendas_loja_dia.groupby(r'Código Venda').sum(numeric_only=True)
    ticket_medio_dia = ticket_medio_dia['Valor Final'].mean()
    # Configurações do e-mail
    nome = df_emails.loc[df_emails['Loja']==loja,'Gerente'].values[0]
    para = df_emails.loc[df_emails['Loja']==loja,'E-mail'].values[0]
    assunto = f'OnePage dia {dia_indicador.day}/{dia_indicador.month} - Loja{loja}'
    if faturamento_dia >=meta_faturamento_dia:
        cor_fat_dia = "green"
    else:
        cor_fat_dia = "red"
    if faturamento_ano >=faturamento_ano:
        cor_fat_ano = "green"
    else:
        cor_fat_ano = "red"
    if qtd_produtos_dia >=qtd_produtos_dia:
        cor_qtd_dia = "green"
    else:
        cor_qtd_dia = "red"
    if qtd_produtos_ano >=qtd_produtos_ano:
        cor_qtd_ano = "green"
    else:
        cor_qtd_ano = "red"
    if ticket_medio_dia >=ticket_medio_dia:
        cor_tkm_dia = "green"
    else:
        cor_tkm_dia = "red"
    if ticket_medio_ano >=ticket_medio_ano:
        cor_tkm_ano = "green"
    else:
        cor_tkm_ano = "red"


    mensagem = f'''
    <p>Bom dia,{nome}</p>
    <p> O resultado de ontem <strong> ({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:
    <table style = "text-align:center">
    <tr>
        <th>Indicador</th>
        <th>Valor dia</th>
        <th>Meta dia</th>
        <th>Cenário dia</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td>{faturamento_dia:.2f}</td>
        <td>{meta_faturamento_dia:.2f}</td>
        <td><font color={cor_fat_dia}>◙</font></td>
    </tr>
    <tr>
        <td>Diversidade Produtos</td>
        <td>{qtd_produtos_dia}</td>
        <td>{meta_qtd_produtos_dia}</td>
        <td><font color={cor_qtd_dia}>◙</font></td>
    </tr>
    <tr>
        <td>Ticket Médio</td>
        <td>{ticket_medio_dia}</td>
        <td>{meta_ticketmedio_dia}</td>
        <td><font color={cor_tkm_dia}>◙</font></td>
    </tr>
    </table>
    <br>
    <table style = "text-align:center">
    <tr>
        <th>Indicador</th>
        <th>Valor ano</th>
        <th>Meta ano</th>
        <th>Cenário ano</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td>{faturamento_ano:.2f}</td>
        <td>{meta_faturamento_ano:.2f}</td>
        <td><font color={cor_fat_ano}>◙</font></td>
    </tr>
    <tr>
        <td>Diversidade Produtos</td>
        <td>{qtd_produtos_ano}</td>
        <td>{meta_qtd_produtos_ano}</td>
        <td><font color={cor_qtd_ano}>◙</font></td>
    </tr>
    <tr>
        <td>Ticket Médio</td>
        <td>{ticket_medio_ano:.2f}</td>
        <td>{meta_ticketmedio_ano:.2f}</td>
        <td><font color={cor_tkm_ano}>◙</font></td>
    </tr>
    </table>
    <p>Segue em aenxo a planilha com todos os dados para mais detalhes </p>

    <p>Qualquer duvida, estou a disposição.</p>
    <p>Att.. Kaio </p>

    '''

    # Cria o objeto MIMEMultipart
    email = MIMEMultipart()
    email['From'] = login
    email['To'] = para
    email['Subject'] = assunto


    # Adiciona o corpo do e-mail
    email.attach(MIMEText(mensagem, 'html'))
    caminho = pathlib.Path.cwd() / path_backup / loja / f'{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx'
    try:
        with open(caminho, 'rb') as arquivo:
            parte_anexo = MIMEBase('application', 'octet-stream')
            parte_anexo.set_payload(arquivo.read())
        # Codifica o anexo em Base64
        encoders.encode_base64(parte_anexo)
        # Define o cabeçalho para o anexo
        parte_anexo.add_header(
            'Content-Disposition',
            f'attachment; filename={f'{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx'}'
        )
        # Anexa a parte do arquivo ao e-mail
        email.attach(parte_anexo)
    except Exception as e:
        print(f"Erro ao anexar o arquivo: {e}")


    # Envia o e-mail usando o servidor SMTP
    try:
        servidor = smtplib.SMTP(smtp_host, smtp_port)
        servidor.starttls()  # Ativa a criptografia TLS
        servidor.login(login, senha)
        servidor.sendmail(login, para, email.as_string())
        servidor.quit()
        print(f"E-mail enviado com sucesso para {para}")
    except Exception as e:
        print(f"Erro ao enviar e-mail: {e}")
    if i ==10:
        break

E-mail enviado com sucesso para kaio.alves123+Helena@gmail.com
E-mail enviado com sucesso para kaio.alves123+Alice@gmail.com
E-mail enviado com sucesso para kaio.alves123+Laura@gmail.com
E-mail enviado com sucesso para kaio.alves123+Manuela@gmail.com
E-mail enviado com sucesso para kaio.alves123+Valentina@gmail.com
E-mail enviado com sucesso para kaio.alves123+Sophia@gmail.com
E-mail enviado com sucesso para kaio.alves123+Isabella@gmail.com
E-mail enviado com sucesso para kaio.alves123+Heloisa@gmail.com


KeyboardInterrupt: 

### Passo 5 - Enviar por e-mail para o gerente

In [154]:

# Configurações do e-mail
nome = df_emails.loc[df_emails['Loja']==loja,'Gerente'].values[0]
para = df_emails.loc[df_emails['Loja']==loja,'E-mail'].values[0]
assunto = f'OnePage dia {dia_indicador.day}/{dia_indicador.month} - Loja{loja}'
if faturamento_dia >=meta_faturamento_dia:
  cor_fat_dia = "green"
else:
  cor_fat_dia = "red"
if faturamento_ano >=faturamento_ano:
  cor_fat_ano = "green"
else:
  cor_fat_ano = "red"
if qtd_produtos_dia >=qtd_produtos_dia:
  cor_qtd_dia = "green"
else:
  cor_qtd_dia = "red"
if qtd_produtos_ano >=qtd_produtos_ano:
  cor_qtd_ano = "green"
else:
  cor_qtd_ano = "red"
if ticket_medio_dia >=ticket_medio_dia:
  cor_tkm_dia = "green"
else:
  cor_tkm_dia = "red"
if ticket_medio_ano >=ticket_medio_ano:
  cor_tkm_ano = "green"
else:
  cor_tkm_ano = "red"


mensagem = f'''
<p>Bom dia,{nome}</p>
<p> O resultado de ontem <strong> ({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:
<table style = "text-align:center">
  <tr>
    <th>Indicador</th>
    <th>Valor dia</th>
    <th>Meta dia</th>
    <th>Cenário dia</th>
  </tr>
  <tr>
    <td>Faturamento</td>
    <td>{faturamento_dia:.2f}</td>
    <td>{meta_faturamento_dia:.2f}</td>
    <td><font color={cor_fat_dia}>◙</font></td>
  </tr>
  <tr>
    <td>Diversidade Produtos</td>
    <td>{qtd_produtos_dia}</td>
    <td>{meta_qtd_produtos_dia}</td>
    <td><font color={cor_qtd_dia}>◙</font></td>
  </tr>
  <tr>
    <td>Ticket Médio</td>
    <td>{ticket_medio_dia}</td>
    <td>{meta_ticketmedio_dia}</td>
    <td><font color={cor_tkm_dia}>◙</font></td>
  </tr>
</table>
<br>
<table style = "text-align:center">
  <tr>
    <th>Indicador</th>
    <th>Valor ano</th>
    <th>Meta ano</th>
    <th>Cenário ano</th>
  </tr>
  <tr>
    <td>Faturamento</td>
    <td>{faturamento_ano:.2f}</td>
    <td>{meta_faturamento_ano:.2f}</td>
    <td><font color={cor_fat_ano}>◙</font></td>
  </tr>
  <tr>
    <td>Diversidade Produtos</td>
    <td>{qtd_produtos_ano}</td>
    <td>{meta_qtd_produtos_ano}</td>
    <td><font color={cor_qtd_ano}>◙</font></td>
  </tr>
  <tr>
    <td>Ticket Médio</td>
    <td>{ticket_medio_ano:.2f}</td>
    <td>{meta_ticketmedio_ano:.2f}</td>
    <td><font color={cor_tkm_ano}>◙</font></td>
  </tr>
</table>
<p>Segue em aenxo a planilha com todos os dados para mais detalhes </p>

<p>Qualquer duvida, estou a disposição.</p>
<p>Att.. Kaio </p>

'''

# Cria o objeto MIMEMultipart
email = MIMEMultipart()
email['From'] = login
email['To'] = para
email['Subject'] = assunto


# Adiciona o corpo do e-mail
email.attach(MIMEText(mensagem, 'html'))
caminho = pathlib.Path.cwd() / path_backup / loja / f'{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx'
try:
    with open(caminho, 'rb') as arquivo:
        parte_anexo = MIMEBase('application', 'octet-stream')
        parte_anexo.set_payload(arquivo.read())
    # Codifica o anexo em Base64
    encoders.encode_base64(parte_anexo)
    # Define o cabeçalho para o anexo
    parte_anexo.add_header(
        'Content-Disposition',
        f'attachment; filename={f'{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx'}'
    )
    # Anexa a parte do arquivo ao e-mail
    email.attach(parte_anexo)
except Exception as e:
    print(f"Erro ao anexar o arquivo: {e}")


# Envia o e-mail usando o servidor SMTP
try:
    servidor = smtplib.SMTP(smtp_host, smtp_port)
    servidor.starttls()  # Ativa a criptografia TLS
    servidor.login(login, senha)
    servidor.sendmail(login, para, email.as_string())
    servidor.quit()
    print("E-mail enviado com sucesso!")
except Exception as e:
    print(f"Erro ao enviar e-mail: {e}")

E-mail enviado com sucesso!


### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

In [14]:
df_fat_lojas = df_vendas[['Loja','Valor Final']].groupby('Loja').sum(numeric_only=True)
df_fat_lojas_ano = df_fat_lojas.sort_values(by="Valor Final",ascending=False)
display(df_fat_lojas_ano)

nome_arquivo = f'{dia_indicador.day}_{dia_indicador.month}_Ranking_anual.xlsx'
path_arquivo = path_backup / nome_arquivo
df_fat_lojas_ano.to_excel(path_arquivo)

df_vendas_dia = df_vendas.loc[df_vendas['Data']==dia_indicador, :]
df_fat_lojas_dia = df_vendas_dia[['Loja','Valor Final']].groupby('Loja').sum(numeric_only=True)
df_fat_lojas_dia = df_fat_lojas_dia.sort_values(by="Valor Final",ascending=False)
display(df_fat_lojas_dia)


nome_arquivo = f'{dia_indicador.day}_{dia_indicador.month}_Ranking_dia.xlsx'
path_arquivo = path_backup / nome_arquivo
df_fat_lojas_dia.to_excel(path_arquivo)


,Valor Final
Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


,Valor Final
Loja,
Salvador Shopping,3950
Novo Shopping Ribeirão Preto,3400
Center Shopping Uberlândia,2651
Shopping Eldorado,2391
Shopping Center Interlagos,1582
Shopping Recife,1366
Norte Shopping,1259
Shopping União de Osasco,1207
Shopping Vila Velha,937


### Passo 8 - Enviar e-mail para diretoria

In [17]:

# Configurações do e-mail
nome = df_emails.loc[df_emails['Loja']=="Diretoria",'Gerente'].values[0]
para = df_emails.loc[df_emails['Loja']=="Diretoria",'E-mail'].values[0]
assunto = f'Ranking lojas {dia_indicador.day}/{dia_indicador.month}'



# Cria o objeto MIMEMultipart
email = MIMEMultipart()
email['From'] = login
email['To'] = para
email['Subject'] = assunto
mensagem = """
Prezados, bom dia!


Segue em anexo os rankings do dia e o anual.


Qualquer dúvida, estou a disposição.
"""

# Adiciona o corpo do e-mail
email.attach(MIMEText(mensagem, 'plain'))
caminho = pathlib.Path.cwd() / path_backup / loja / f'{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx'
try:
    with open(caminho, 'rb') as arquivo:
        parte_anexo = MIMEBase('application', 'octet-stream')
        parte_anexo.set_payload(arquivo.read())
    # Codifica o anexo em Base64
    encoders.encode_base64(parte_anexo)
    # Define o cabeçalho para o anexo
    parte_anexo.add_header(
        'Content-Disposition',
        f'attachment; filename={f'{dia_indicador.day}_{dia_indicador.month}_Ranking_anual.xlsx'}'
    )
    parte_anexo.add_header(
        'Content-Disposition',
        f'attachment; filename={f'{dia_indicador.day}_{dia_indicador.month}_Ranking_dia.xlsx'}'
    )
    # Anexa a parte do arquivo ao e-mail
    email.attach(parte_anexo)
except Exception as e:
    print(f"Erro ao anexar o arquivo: {e}")


# Envia o e-mail usando o servidor SMTP
try:
    servidor = smtplib.SMTP(smtp_host, smtp_port)
    servidor.starttls()  # Ativa a criptografia TLS
    servidor.login(login, senha)
    servidor.sendmail(login, para, email.as_string())
    servidor.quit()
    print(f"E-mail enviado com sucesso para {para}")
except Exception as e:
    print(f"Erro ao enviar e-mail: {e}")
  

E-mail enviado com sucesso para kaio.alves123+Diretoria@gmail.com
